# Example: fine-tuning 1B model on CNN Dailymail summarization

In [1]:
!hostname
!pwd

t1v-n-d84a2c81-w-0


/home/patflick/dev/git/maxtext/MaxText


In [2]:
my_bucket = "gs://patflick-maxtext-lingvo"
base_output_directory= my_bucket + "/maxtext/2b/20231121"

checkpoint_path = "gs://test-example-123/maxtest-lg/2b-c4/v4-32/maxtext_cnndm/checkpoints"
# to continue training this run 
checkpoint_path = "gs://patflick-maxtext-lingvo/maxtext/2b/20231121/finetuning-2b-20k-v2/checkpoints"

config_path = "config/base-2b.yml"
# Modify the base config:
extra_config = {
    # Run config
    "run_name": "finetuning-2b-20k-v2",
    "base_output_directory": base_output_directory,
    "save_period": 5000,

    # Load common crawl checkpoint
    "load_from_other_directory": checkpoint_path,
    #"load_from_other_directory_step": 12000,

    # Parallelism and KV config
    "dcn_tensor_parallelism": 4,
    "ici_tensor_parallelism": 4,
    "enable_flash_attention": False,

    # use CNN Dailymail summarization dataset
    "dataset_type": "cnndm",

    # Re-use same tokenizer (this currently doesn't work!)
    "assets_path": "/home/patflick/dev/git/maxtext/assets", # TODO: move assets to GCS

    # Config for `training`
    "learning_rate": 3.e-5,
    "steps": 40000,
}

import pyconfig
pyconfig.initialize(["", "configs/base-2b.yml"] + [f"{k}={v}" for k,v in extra_config.items()])
config = pyconfig.config
pyconfig._config.keys

OrderedDict([('run_name', 'finetuning-2b-20k-v2'),
             ('load_parameters_path', ''),
             ('load_from_other_directory',
              'gs://patflick-maxtext-lingvo/maxtext/2b/20231121/finetuning-2b-20k-v2/checkpoints'),
             ('load_from_other_directory_step', -1),
             ('reuse_example_batch', 0),
             ('metrics_file', ''),
             ('gcs_metrics', False),
             ('dtype', dtype(bfloat16)),
             ('int8_training', False),
             ('use_dqdg', False),
             ('fwd_int8', True),
             ('dlhs_int8', True),
             ('drhs_int8', False),
             ('fwd_int8_qk', False),
             ('dlhs_int8_qk', False),
             ('drhs_int8_qk', False),
             ('fwd_int8_pv', True),
             ('dlhs_int8_pv', True),
             ('drhs_int8_pv', False),
             ('aqt_use_dummy_static_bound', False),
             ('aqt_use_fwd_quant', False),
             ('aqt_rng_type', 'jax.uniform'),
             ('g

In [3]:
import train
import os
import jax

os.environ["TFDS_DATA_DIR"] = pyconfig.config.dataset_path + "/"
train.train_loop(config) 
jax.distributed.shutdown()

2023-11-21 19:33:14.695567: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/patflick/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Creating checkpoint manager...
Checkpoint manager created!
Devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)] (num_devices: 4)
Decided on mesh: [[[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)
   TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0)
   TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0)
   TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)]]]


restoring state from this run's directory latest step         20000


I0000 00:00:1700595202.895581 1022249 gcs_resource.cc:99] Using default AdmissionQueue with limit 32
I0000 00:00:1700595202.899154 1024651 google_auth_provider.cc:179] Running on GCE, using service account 903354779218-compute@developer.gserviceaccount.com


number parameters: 2.097 billion
Per train step, total TFLOPs will be 348.83, split as 88.65% learnable weight flops and 11.35% attention flops
completed step: 20001, seconds: 18.073, TFLOP/s/device: 19.302, loss: 0.037
completed step: 20002, seconds: 0.762, TFLOP/s/device: 458.027, loss: 0.072
completed step: 20003, seconds: 0.499, TFLOP/s/device: 698.419, loss: 0.052
completed step: 20004, seconds: 0.500, TFLOP/s/device: 698.318, loss: 0.046
completed step: 20005, seconds: 0.500, TFLOP/s/device: 697.984, loss: 0.058
completed step: 20006, seconds: 0.499, TFLOP/s/device: 698.389, loss: 0.033
completed step: 20007, seconds: 0.500, TFLOP/s/device: 698.086, loss: 0.056
completed step: 20008, seconds: 0.500, TFLOP/s/device: 697.390, loss: 0.053
completed step: 20009, seconds: 0.499, TFLOP/s/device: 698.575, loss: 0.051
completed step: 20010, seconds: 0.499, TFLOP/s/device: 698.575, loss: 0.043
completed step: 20011, seconds: 0.500, TFLOP/s/device: 698.289, loss: 0.048
completed step: 2001

In [8]:
# 
import jax
jax.distributed.shutdown()